In [22]:
import gc
import h5py
import quimb as qu
import quimb.tensor as qtn
import numpy as np
import pandas as pd
from sklearn import svm
from sklearn.metrics import *
import matplotlib.pyplot as plt
from Haldane_anis_class import *

In [2]:
L = 7  # number of particle which has to be odd for calculate faster
ls = 10 # the scale of dividing the range classical parameters

In [3]:
train_set = Haldan_anis(L = L, ls = ls).generate_train_set() #ts[0] = DMRG_state, ts[1] = DMRG_target, ts[2] = points
gc.collect(2)

/home/mehran/4th_project-Haldan_anisotropic/Haldane_anis_class.py:23: ComplexWarning: Casting complex values to real discards the imaginary part
  W[0, 0, :, :] = I
/home/mehran/4th_project-Haldan_anisotropic/Haldane_anis_class.py:24: ComplexWarning: Casting complex values to real discards the imaginary part
  W[0, 1, :, :] = res[0]+res[1]+res[2]
/home/mehran/4th_project-Haldan_anisotropic/Haldane_anis_class.py:25: ComplexWarning: Casting complex values to real discards the imaginary part
  W[0, 4, :, :] = (d*Z@Z)+(e*X@X)-(e*Y@Y)
/home/mehran/4th_project-Haldan_anisotropic/Haldane_anis_class.py:26: ComplexWarning: Casting complex values to real discards the imaginary part
  W[1, 4, :, :] = I
/home/mehran/4th_project-Haldan_anisotropic/Haldane_anis_class.py:27: ComplexWarning: Casting complex values to real discards the imaginary part
  W[4, 4, :, :] = I
/home/mehran/4th_project-Haldan_anisotropic/Haldane_anis_class.py:23: ComplexWarning: Casting complex values to real discards the imag

0

In [14]:
with h5py.File("train_set_DMRG_target.hdf5", "w") as f:
    f.create_dataset("DMRG_target", data=np.array(train_set[3]))

In [5]:
test_set = Haldan_anis(L = L, ls = ls).generate_test_set()#ts[0]=DMRG_state, ts[1]=points

/home/mehran/4th_project-Haldan_anisotropic/Haldane_anis_class.py:23: ComplexWarning: Casting complex values to real discards the imaginary part
  W[0, 0, :, :] = I
/home/mehran/4th_project-Haldan_anisotropic/Haldane_anis_class.py:24: ComplexWarning: Casting complex values to real discards the imaginary part
  W[0, 1, :, :] = res[0]+res[1]+res[2]
/home/mehran/4th_project-Haldan_anisotropic/Haldane_anis_class.py:25: ComplexWarning: Casting complex values to real discards the imaginary part
  W[0, 4, :, :] = (d*Z@Z)+(e*X@X)-(e*Y@Y)
/home/mehran/4th_project-Haldan_anisotropic/Haldane_anis_class.py:26: ComplexWarning: Casting complex values to real discards the imaginary part
  W[1, 4, :, :] = I
/home/mehran/4th_project-Haldan_anisotropic/Haldane_anis_class.py:27: ComplexWarning: Casting complex values to real discards the imaginary part
  W[4, 4, :, :] = I
/home/mehran/4th_project-Haldan_anisotropic/Haldane_anis_class.py:23: ComplexWarning: Casting complex values to real discards the imag

In [29]:
with h5py.File("test_set_DMRG.hdf5", "w") as f:
    f.create_dataset("points", data = np.array(test_set[1]))

In [7]:
def gram_train():
    Xt = train_set[2]
    d = len(Xt)
    gram_matrix_train = np.zeros((d,d))
    for i in range(d):
        for j in range(d):
            gram_matrix_train[i,j] = np.abs(Xt[i].H @ Xt[j])**2/ (np.abs(Xt[i].H @ Xt[i])*np.abs(Xt[j].H @ Xt[j]))
    return gram_matrix_train

gtrain = gram_train()

with h5py.File("gram_train.hdf5", "w") as f:
    f.create_dataset("gram_train", data = gram_train())

gc.collect(2)

0

In [10]:
gtrain.shape

(764, 764)

In [8]:
def gram_test():
    Xte = test_set[0]
    d1 = len(Xte)
    Xtr = train_set[2]
    d2 = len(Xtr)
    gram_matrix_test = np.zeros((d1,d2))
    for i in range(d1):
        for j in range(d2):
            gram_matrix_test[i,j] = np.abs((Xte[i].H @ Xtr[j]))**2/ (np.abs((Xte[i].H @ Xte[i]))*np.abs((Xtr[j].H @ Xtr[j])))
    return gram_matrix_test

gtest = gram_test()

with h5py.File("gram_test.hdf5", "w") as f:
    f.create_dataset("gram_test", data = gram_test())

gc.collect(2)

0

In [31]:
clf = svm.SVC(kernel='precomputed')
y = train_set[3]
clf.fit(gtrain, y) 
y_pred = clf.predict(gtrain)

In [33]:
y_pred

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,